In [9]:
import pickle
import numpy as np
import torch

from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy
from cs285.infrastructure import pytorch_util as ptu

In [10]:
import torch
print(torch.__version__)
print(torch.version.cuda)   # CUDA version that PyTorch was built with
print(torch.backends.cudnn.version())


2.7.1+cu118
11.8
90100


In [11]:
torch.cuda.is_available()

True

In [12]:
# import numpy as np
# import torch
# from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy
# from cs285.infrastructure import pytorch_util as ptu

# # init device (CPU is fine)
# ptu.init_gpu(use_gpu=False)

# # load expert into a normal PyTorch nn.Module
# expert = LoadedGaussianPolicy("../policies/experts/Ant.pkl").to(ptu.device)

# # show layer names and shapes
# sd = expert.state_dict()
# for k, v in sd.items():
#     print(f"{k:35s} {tuple(v.shape)}")

# # peek at a few actual numbers (first row of output layer)
# W = sd["output_layer.weight"]        # shape (8, 64) for Ant
# b = sd["output_layer.bias"]          # shape (8,)
# print("\noutput_layer.weight[0, :5] =", W[0, :5])
# print("output_layer.bias[:5]        =", b[:5])

# # sanity: obs → action
# obs = np.random.randn(1, 111).astype(np.float32)
# act = expert.get_action(obs)
# print("\nAction shape:", act.shape, "first action:", act[0])


In [13]:
import pickle

ckpt = pickle.load(open("../policies/experts/Ant.pkl", "rb"))
print("Top-level keys:", ckpt.keys())

gp = ckpt["GaussianPolicy"]
print("GaussianPolicy keys:", gp.keys())

# Don't index gp['hidden'] like a list; it's a dict:
hidden = gp["hidden"]
print("type(hidden):", type(hidden))
print("hidden keys:", getattr(hidden, "keys", lambda: [])())

out = gp["out"]
print("type(out):", type(out))
print("out keys:", getattr(out, "keys", lambda: [])())

print("logstdevs_1_Da shape:", getattr(gp["logstdevs_1_Da"], "shape", None))

obsnorm = gp["obsnorm"]
print("obsnorm keys:", getattr(obsnorm, "keys", lambda: [])())


Top-level keys: dict_keys(['GaussianPolicy', 'nonlin_type'])
GaussianPolicy keys: dict_keys(['hidden', 'obsnorm', 'logstdevs_1_Da', 'out'])
type(hidden): <class 'dict'>
hidden keys: dict_keys(['FeedforwardNet'])
type(out): <class 'dict'>
out keys: dict_keys(['AffineLayer'])
logstdevs_1_Da shape: (1, 8)
obsnorm keys: dict_keys(['Standardizer'])


In [14]:
obs_dim = 111  # for Ant-v4, may vary by env
fake_obs = np.random.randn(1, obs_dim).astype(np.float32)
act = expert.get_action(fake_obs)
print("Sample action from expert:", act)
print("Action shape:", act.shape)

Sample action from expert: [[ 1.0613871  -0.36487013 -1.4425466  -0.56970346  0.5854364   0.6475107
  -0.25946915  0.6565325 ]]
Action shape: (1, 8)


In [22]:
expert_data_file = "../expert_data/expert_data_Ant-v4.pkl"
with open(expert_data_file, "rb") as f:
    expert_paths = pickle.load(f)

C:\Users\Yumlum\AppData\Local\Temp\ipykernel_15676\635976527.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  expert_paths = pickle.load(f)


In [24]:
# ---- Load the expert dataset (trajectories) ----
expert_data_file = "../expert_data/expert_data_Ant-v4.pkl"
with open(expert_data_file, "rb") as f:
    expert_paths = pickle.load(f)

print("Number of expert trajectories:", len(expert_paths))
first_path = expert_paths[0]
print("Keys in one path:", first_path.keys())
print("Obs shape:", first_path["observation"].shape)
print("Act shape:", first_path["action"].shape)
print("Reward shape:", first_path["reward"].shape)

# peek at some values
print("First obs[0]:", first_path["observation"][0][:5])  # first 5 dims
print("First act[0]:", first_path["action"][0])
print("First reward[0]:", first_path["reward"][0])

Number of expert trajectories: 2
Keys in one path: dict_keys(['observation', 'action', 'reward', 'next_observation', 'terminal'])
Obs shape: (1000, 27)
Act shape: (1000, 8)
Reward shape: (1000,)
First obs[0]: [ 0.82588086  0.99502704 -0.08480735 -0.04968133 -0.01614537]
First act[0]: [ 0.29435965  0.01958522 -1.1779833  -0.01043977 -0.30546713  0.11675249
  0.7819461   0.19286692]
First reward[0]: -0.5191441852876566


C:\Users\Yumlum\AppData\Local\Temp\ipykernel_15676\3661943778.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  expert_paths = pickle.load(f)


In [27]:
import gym
env = gym.make("Ant-v4")
print("env obs dim:", env.observation_space.shape)  # expect (111,)
print("env act dim:", env.action_space.shape)       # expect (8,)

# 2) What does your expert policy expect (from state_dict)?
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy
from cs285.infrastructure import pytorch_util as ptu
ptu.init_gpu(use_gpu=False)
expert = LoadedGaussianPolicy("../policies/experts/Ant.pkl").to(ptu.device)
sd = expert.state_dict()
print("first hidden weight shape:", sd["hidden_layers.0.weight"].shape)  # (64, 111) if Ant-v4
print("output weight shape:", sd["output_layer.weight"].shape)           # (8, 64)

# 3) What does your dataset say?
import pickle
paths = pickle.load(open("../expert_data/expert_data_Ant-v4.pkl", "rb"))
print("dataset obs dim:", paths[0]["observation"].shape[1])  # you saw 27
print("dataset act dim:", paths[0]["action"].shape[1])       # 8

env obs dim: (27,)
env act dim: (8,)
GPU not detected. Defaulting to CPU.
obs (1, 111) (1, 111)
first hidden weight shape: torch.Size([64, 111])
output weight shape: torch.Size([8, 64])
dataset obs dim: 27
dataset act dim: 8


c:\Users\Yumlum\.conda\envs\rl\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\Yumlum\.conda\envs\rl\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\Yumlum\AppData\Local\Temp\ipykernel_15676\3830237530.py:17: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access fu

In [1]:
import pickle
import numpy as np
import pandas as pd

# point this to YOUR file
EXPERT_DATA = "../expert_data/expert_data_Ant-v4.pkl"  # adjust path if needed

In [2]:
with open(EXPERT_DATA, "rb") as f:
    paths = pickle.load(f)

print("Number of trajectories:", len(paths))
first = paths[0]
print("Keys:", first.keys())
print("Obs shape:", first["observation"].shape)
print("Act shape:", first["action"].shape)
print("Reward shape:", first["reward"].shape)

# peek a few values
print("obs[0][:5]:", first["observation"][0][:5])
print("act[0]    :", first["action"][0])
print("rew[0]    :", first["reward"][0])

Number of trajectories: 2
Keys: dict_keys(['observation', 'action', 'reward', 'next_observation', 'terminal'])
Obs shape: (1000, 27)
Act shape: (1000, 8)
Reward shape: (1000,)
obs[0][:5]: [ 0.82588086  0.99502704 -0.08480735 -0.04968133 -0.01614537]
act[0]    : [ 0.29435965  0.01958522 -1.1779833  -0.01043977 -0.30546713  0.11675249
  0.7819461   0.19286692]
rew[0]    : -0.5191441852876566


(1000, 27)

In [3]:
from cs285.infrastructure.utils import convert_listofrollouts
from cs285.infrastructure.replay_buffer import ReplayBuffer

c:\Users\Yumlum\.conda\envs\rl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
buf = ReplayBuffer(max_size=1_000_000)

print("len(buf) before add:", len(buf))
buf.add_rollouts(paths, concat_rew=True)   # flatten and append
print("len(buf) after  add:", len(buf))

print("obs:", buf.obs.shape, "  acs:", buf.acs.shape,
      "  rews:", buf.rews.shape if not isinstance(buf.rews, list) else f"list({len(buf.rews)})",
      "  next_obs:", buf.next_obs.shape, "  terminals:", buf.terminals.shape)

len(buf) before add: 0
(2000, 27)
len(buf) after  add: 2000
obs: (2000, 27)   acs: (2000, 8)   rews: (2000,)   next_obs: (2000, 27)   terminals: (2000,)


In [5]:
import torch


In [6]:
torch.cuda.is_available()

False